In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import tensorflow_datasets as tfds

# Chargement des images et des étiquettes
(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
    "tf_flowers",
    split=["train[:70%]", "train[70%:100%]"],  # Correction de la division d'entraînement et de test
    batch_size=-1,
    as_supervised=True  # Inclure les étiquettes
)

# Redimensionnement des images
train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

# Transformation des étiquettes au format correct
train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

# Chargement du modèle VGG16
base_model = VGG16(weights="imagenet", include_top=False, input_shape=train_ds.shape[1:])  # Correction de la forme d'entrée

base_model.trainable = False  # Poids non entraînables

# Prétraitement des entrées
train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

base_model.summary()

# Construction du modèle
flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')

model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

# Compilation et configuration du modèle
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

# Entraînement du modèle
model.fit(train_ds, train_labels, epochs=100, validation_split=0.2, batch_size=32, callbacks=[es])


model.summary()
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(test_ds, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Prédiction sur l'ensemble de test
predictions = model.predict(test_ds)

# Affichage de quelques prédictions
for i in range(5):
    print("Image", i+1)
    print("Predicted label:", predictions[i])
    print("True label:", test_labels[i])
    print()
# Évaluation du modèle sur l'ensemble de test
test_loss, test_accuracy = model.evaluate(test_ds, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Prédiction sur l'ensemble de test
predictions = model.predict(test_ds)

# Affichage de quelques prédictions
for i in range(5):
    print("Image", i+1)
    print("Predicted label:", predictions[i])
    print("True label:", test_labels[i])


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 